In [3]:
import tensorflow as tf
import numpy as np
tf.get_logger().setLevel("ERROR")

2022-04-07 21:11:22.296838: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-07 21:11:22.296880: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
y_pred = tf.constant([[1, 0.5, 0.4, 0.4], [2, 0.3, 0.1, 0.4]])
y_true = tf.constant([[1.], [1.8]])

2022-04-07 21:11:24.856407: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-07 21:11:24.856629: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-07 21:11:24.856642: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-07 21:11:24.856666: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (clinton-ThinkPad-T450): /proc/driver/nvidia/version does not exist
2022-04-07 21:11:24.856927: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow wi

In [5]:
def prepare_evidential_output(y_true, y_pred):
    if y_true.shape.rank == 1:
        y_true = tf.expand_dims(y_true, axis=-1)
    if y_pred.shape.rank == 1:
        y_pred = tf.expand_dims(y_pred, axis=0)
    assert y_pred.shape[1] == 4, f"Expects four parameters for the evidential output instead got {y_pred.shape[1]}"

    gamma, lamb, alpha, beta = tf.split(y_pred, 4, axis=-1)
    
    return y_true, gamma, lamb, alpha, beta

def normal_inverse_gamma_negative_log_likelihood(y_true, gamma, lamb, alpha, beta):

    negative_log_likelihood = -(
            tf.math.log(2**(0.5 + alpha))
            + tf.math.log(beta**alpha)
            + 0.5 *tf.math.log(lamb/ (2 * np.math.pi * ( 1 + lamb)))
            - (0.5 + alpha) * tf.math.log(2 * beta + lamb * (gamma - y_true)**2 / (1 + lamb))
            # + tf.math.lgamma(alpha)
            # - tf.math.lgamma(alpha+0.5)
        )
    return negative_log_likelihood

def normal_inverse_gamma_sum_of_squares(y_true, gamma, lamb, alpha, beta):

    log_likelihood = (
        tf.math.log(beta*(1 + lamb)/lamb + (alpha - 1)*(y_true - gamma)**2)
        + tf.math.lgamma(alpha - 1)
        - tf.math.lgamma(alpha)
    )

    return log_likelihood

def normal_inverse_gamma_regularizer(y_true, gamma, lamb, alpha):
    return tf.abs(y_true - gamma) * (2*alpha + lamb)

def evidential_regression_loss(y_true, y_pred, coeff=1.0, loss_fn=normal_inverse_gamma_sum_of_squares):
    if y_true.shape.rank == 1:
        y_true = tf.expand_dims(y_true, axis=-1)
    if y_pred.shape.rank == 1:
        y_pred = tf.expand_dims(y_pred, axis=0)
    assert y_pred.shape[1] == 4, f"Expects four parameters for the evidential output instead got {y_pred.shape[1]}"

    gamma, lamb, alpha, beta = tf.split(y_pred, 4, axis=-1)

    loss = tf.reduce_mean(loss_fn(y_true, gamma, lamb, alpha, beta))

    regularizer = tf.reduce_mean(normal_inverse_gamma_regularizer(y_true, gamma, lamb, alpha))

    return loss + coeff * regularizer



In [6]:
y_pred = tf.constant([[1, 0.5, 0.4, 0.3], [2, 0.3, 0.1, 0.7]])
y_true = tf.constant([[1.], [1.8]])
nll_reg_mean = 1.1719854
nll_reg_mean_gamma = 2.4643755
nll_sos_mean = 0.85427415

In [7]:
def NIG_NLL(y, y_pred):
    y, gamma, v, alpha, beta = prepare_evidential_output(y_true, y_pred)
    twoBlambda = 2*beta*(1+v)

    nll = 0.5*tf.math.log(np.pi/v)  \
        - alpha*tf.math.log(twoBlambda)  \
        + (alpha+0.5) * tf.math.log(v*(y-gamma)**2 + twoBlambda)  \
        # + tf.math.lgamma(alpha)  \
        # - tf.math.lgamma(alpha+0.5)
    return nll

tf.reduce_mean(NIG_NLL(y_true, y_pred))

<tf.Tensor: shape=(), dtype=float32, numpy=1.1719854>

In [8]:
y_true, gamma, v, alpha, beta = prepare_evidential_output(y_true, y_pred)
normal_inverse_gamma_negative_log_likelihood(y_true, gamma, v, alpha, beta)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.8662584],
       [1.4777125]], dtype=float32)>

In [9]:
normal_inverse_gamma_sum_of_squares(y_true, gamma, v, alpha, beta)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.4054652],
       [1.203083 ]], dtype=float32)>

In [14]:

def deep_evidential_regression_loss(
    y_true, 
    y_pred, 
    coeff: float = 1, 
    loss_fn = normal_inverse_gamma_sum_of_squares
):
    '''
    Maximum likelihood objective for deep evidential regression model using negative 
    log likelihood or sum of squares loss of the normal inverse gamma distribution.

    :param y_true: The output variable values.
    :param y_pred: The four output parameters of the deep evidential regression model
        that characterize the normal inverse gamma distribution given in the order: gamma,
        lambda, alpha, beta.
    :param coeff: Regularization weight coefficient.
    :param loss_fn: The base loss function used. By default we use the recommended loss function
        based on the sum of squares. An alternate loss function is also defined based on the negative
        log likelihood :function: `~trieste.models.keras.utils.normal_inverse_gamma_negative_log_likelihood`.
    :return: The model evidence values.  
    '''
    if y_true.shape.rank == 1:
        y_true = tf.expand_dims(y_true, axis=-1)
    if y_pred.shape.rank == 1:
        y_pred = tf.expand_dims(y_pred, axis=0)
    tf.debugging.assert_shapes([(y_pred, (y_pred.shape[0], 4))])

    gamma, lamb, alpha, beta = tf.split(y_pred, 4, axis=-1)

    loss = loss_fn(y_true, gamma, lamb, alpha, beta)
    regularization = normal_inverse_gamma_regularizer(y_true, gamma, lamb, alpha)

    return tf.reduce_mean(loss + coeff * regularization, axis=0)
    

In [18]:
y_true = tf.constant([1., 1.8])
y_pred = tf.constant([
                [1., 0.5, 0.4, 0.3],
                [2., 0.3, 0.1, 0.7]
            ])

import numpy.testing as npt
npt.assert_almost_equal(deep_evidential_regression_loss(y_true, y_pred), 0.85427415)

AttributeError: 'tuple' object has no attribute 'rank'